# **Lab 1: Matrix Factorization**
**Mirja Johnsson**

# **Abstract**

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [10]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them.

In [6]:
import numpy as np

# **Introduction**

# **Method**

## Assignment 1.1
In assignment 1.1 the task is to write a matrix-vector multiplication function that takes as its input a real quatratic matrix M in compressed-sparse-row format, and a vector x, and returns their product.

For a future implementation I would change this to handle non-square matrices, but didn't do that now.

In [2]:
#Assignment 1.1
# Input: vector x, sparse (real, quadratic) matrix A: CRS arrays val, col_idx, row_ptr
#Output: matrix-vector product b=Ax
def CSR(row_ptr, col_idx, v, x):
  result = np.zeros(x.size, dtype=float)
  for i in range(row_ptr.size-1):
    r = row_ptr[i+1]-row_ptr[i]
    for j in range(r):
      result[i] += v[j+i]*x[col_idx[j+i]]
  return result


## Assignment 1.2
I implemented algorithm 5.3 from the course book: Q, R = modified_gram_schmidt_iteration(A):

It is specified that the input matrix A will be square and have full rank, so we do not need to verify this in the function. However for the code to be usable in the future it is a good idea to add these checks.

For verification I use the frobenius norm from numpy to verify that the solution is close enough to what it should be.



In [3]:
# Assigment 1.2
# QR-factorization

#Input: (real, quadratic, invertible) matrix A
#Output: orthogonal matrix Q, upper triangular matrix R, such that A=QR
#Test: R upper triangular, Frobenius norms || Q^TQ-I ||_F, || QR-A ||_F

#v[:]for the whole column vector v, just write v in python
#and A[:,j] for the jth column vector of the matrix A.
#Analogously, A[i,:] refers to the ith row vector of A

#(Q, R) = modified_gram_schmidt_iteration(A).
#Input: a full rank n x n matrix A.
#Output: an orthogonal n x n matrix Q and an upper triangular n x n matrix R.
def gram_schmidt_modified(A):
    m,n = A.shape
    if m != n:
        #print("Matrix must be square")
        raise Exception("Matrix must be square")
    if (np.linalg.matrix_rank(A)!= n):
        raise Exception("Matrix must have full rank")
    Q = np.zeros_like(A, dtype = float)
    R = np.zeros_like(A, dtype = float)
    #n = A.shape[0]
    A = np.copy(A)
    #v = np.zeros(n)
    for j in range(n):
        v = A[:,j]
        for i in range(j):
            R[i,j] = scalar_product(Q[:,i], v)
            v -=  R[i,j]*Q[:,i]
        #end for
        R[j,j] = np.linalg.norm(v)
        Q[:,j] = v/R[j,j]
    #end for
    return Q, R


def scalar_product(v1, v2):
    sum = 0
    if len(v1) != len(v2):
        return None
    else:

      for i in range(len(v1)):
          sum += v1[i] * v2[i]
    return sum




Describe the methods you used to solve the problem. This may be a combination of text, mathematical formulas (Latex), algorithms (code), data and output.  

## 1.3
For this part of the assignment we are asked to implement a direct solver for Ax = b, where A is a square matrix and x, b are vectors. A and b are given, and the task is to return x.

Since we have just found a way to factorize a square matrix we can use this to find A^-1 (A inverse) in order to solve for x.
Since A = QR -> Ax = b == QRx = b


QRx = b    ->    Rx = Q^-1b   ->    x = R^-1 Q^-1 b    ->


x = R^-1 Q^-1 b

The inverse of an upper triangular matrix like R can be found by using backward substitution. The inverse of an orthogonal matrix like Q is just its transpose.

In [4]:
#1.3

#ALGORITHM 5.2 from the course book
# Input: an upper triangular matrix U and a vector b. Output: the solution x to the equation Ux = b.
def backward_substitution(U, b):
    x = np.zeros_like(b)
    n = len(x)
    x[n-1] = b[n-1]/U[n-1,n-1]
    for i in range(n-2,-1, -1):
        sum = 0
        for j in range(i+1, n):
            sum = sum + U[i,j]*x[j]
        #end for
        x[i] = (b[i] - sum)/U[i,i]
    #end for
    return x

def direct_solver(A, b):
    Q, R = gram_schmidt_modified(A)
    Qinv = np.transpose(Q)
    #first multiply b with Q^T since the order of multiplication is non-commutative (generally)
    Qinv_b = np.matmul(Qinv, b)
    #solve for x via backward substitution
    x = backward_substitution(R, Qinv_b)
    return x

# **Results**

Present the results. If the result is an algorithm that you have described under the *Methods* section, you can present the data from verification and performance tests in this section. If the result is the output from a computational experiment this is where you present a selection of that data.

# TESTS:
####1.1

I have constructed a 4x4 matrix in both sparse and full representation. We can see that the result of performing the multiplication with vecotr x gives equivalent results when using my CSR function and when using numpy.dot()

In [7]:
rowp = np.array([0,1,2,3,5])
col = np.array([0,1,2,1,3])
v = np.array([4,7,13,6,1], dtype = float)
x = np.array([1,2,2,1], dtype = float)
print("The result of the multiplication using CSR: \n", CSR(rowp,col,v,x), "\n")
fullmatrix = np.array([[4,0,0,0],[0,7,0,0],[0,0,13,0],[0,6,0,1]])
print("\n The result of using numpy's matrix multiplication with the spare matrix \n", fullmatrix.dot(x), "\n")

The result of the multiplication using CSR: 
 [ 4. 14. 26. 13.] 


 The result of using numpy's matrix multiplication with the spare matrix 
 [ 4. 14. 26. 13.] 



#### 1.2
To see that R is upper triangular it is enough to run the gram_schmidt algorithm print R, and look at the results.
To check the frobenius norms I use numpy's norm library and specify that the Frobenius norm should be used. It's actually the default norms for matrices but it doesn't hurt to be clear.



In [8]:
#1.2

if __name__ == "__main__":

    A = np.array(([2,-1,1,0],[0,2,1,0], [0,1,3,0], [0,0,0,5]), dtype = float)
    #A = np.array(([0,0,0,0],[0,2,1,0], [0,1,3,0], [0,0,1,5]), dtype = float)
    Q, R = gram_schmidt_modified(A)
    print("Q: \n", Q, "\n")
    print("R: \n", R, "\n")

    #Check QR - A is  0
    print("[QR] - [A] should be  0")
    print(np.linalg.norm(np.matmul(Q,R)-A , ord = 'fro'), "\n")
    #Check Q Q^T -I
    print("[QQ^T]-I should be very close to 0")
    print(np.linalg.norm(np.matmul(Q,Q.transpose())-np.identity(4), ord = 'fro'))

Q: 
 [[ 1.          0.          0.          0.        ]
 [ 0.          0.89442719 -0.4472136   0.        ]
 [ 0.          0.4472136   0.89442719  0.        ]
 [ 0.          0.          0.          1.        ]] 

R: 
 [[ 2.         -1.          1.          0.        ]
 [ 0.          2.23606798  2.23606798  0.        ]
 [ 0.          0.          2.23606798  0.        ]
 [ 0.          0.          0.          5.        ]] 

[QR] - [A] should be  0
0.0 

[QQ^T]-I should be very close to 0
1.5801635193621484e-16


In [9]:
#1.3
if __name__ == "__main__":

    A = np.array(([2,-1,1,0],[0,2,1,0], [0,1,3,0], [0,0,0,5]), dtype = float)
    b = np.array([0.4,2.0,2.0,3.0], dtype = float)
    x = direct_solver(A, b)
    print("A =  \n", A, "\n")
    print("b = ", b, "\n")
    print("Solved for x: \n")
    print("x = ", x, "\n")


    #return
    b_test = np.dot(A,x)
    print("Test to see that we get the original vector b if we multiply A with x \n")
    print("Test : calculated b: ", b_test, " == original b: ", b)


A =  
 [[ 2. -1.  1.  0.]
 [ 0.  2.  1.  0.]
 [ 0.  1.  3.  0.]
 [ 0.  0.  0.  5.]] 

b =  [0.4 2.  2.  3. ] 

Solved for x: 

x =  [0.4 0.8 0.4 0.6] 

Test to see that we get the original vector b if we multiply A with x 

Test : calculated b:  [0.4 2.  2.  3. ]  == original b:  [0.4 2.  2.  3. ]


# **Discussion**

Some useful code for linear-algebra applications was implemented in this lab. The code that does this is already available in numpy and scipy and other scientific libraries. However I think it is very valuable for us students to implement them ourselves to get a deeper understanding of what is atually happening.